# Random Forest - Implementation

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import os
import optuna
import joblib

/Users/tushar/_main/uni/sem-3/nlp/Project/ProjectNLP/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read data

In [ ]:
file_path = os.path.join("Data", "track-a.csv")
dataframe = pd.read_csv(file_path)
dataframe.head()

## Data Pre-processing

### Stop word removal

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

dataframe['text'] = dataframe['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

### Lemmatization

In [ ]:
word_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
# TODO: Lot of punctuations (remove)

def lemmatize_text(txt):
    out = ""
    for word in word_tokenizer.tokenize(txt):
        out = out + lemmatizer.lemmatize(word) + " "
    return out

dataframe["text"] = dataframe.text.apply(lemmatize_text)



### Train-Test Split

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

X_texts = dataframe['text']
vectorizer = CountVectorizer()
X_texts_vec = vectorizer.fit_transform(X_texts)


emotions = ["anger", "fear", "joy", "sadness", "surprise"]
Y_emotions = dataframe[emotions]


X_train_text, X_test_text, Y_train_labels, Y_test_labels = train_test_split(X_texts_vec, Y_emotions, test_size = 0.1)

## Random Forest Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 25, 500, step = 25)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
    max_depth = trial.suggest_int("max_depth", 2, 64, step = 2)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    random_state = trial.suggest_int("random_state", 0, 100, step = 20)
    warm_start = trial.suggest_categorical("warm_state", [False, True])

    model = RandomForestClassifier(n_estimators = n_estimators, 
                                   criterion = criterion, 
                                   max_depth = max_depth, 
                                   max_features = max_features, 
                                   random_state = random_state, 
                                   warm_start = warm_start)

    model.fit(X_train_text,Y_train_labels)

    yPred = model.predict(X_test_text)
    
    f1score = f1_score(Y_test_labels, yPred, average = "samples", zero_division = 0)
    
    bestf1score = float(0.0)

    if f1score > bestf1score: #type: ignore
        bestf1score = f1score

    return bestf1score

In [ ]:
studyRFC = optuna.create_study(study_name = "RFC", direction = "maximize", sampler = optuna.samplers.TPESampler(seed = 42), storage = "sqlite:///db.sqlite3")

In [2]:
optuna.delete_study(study_name="RFC", storage = "sqlite:///db.sqlite3")

In [ ]:
def _objective(trial):
    return objective(trial)

In [ ]:
studyRFC.optimize(_objective, n_trials = 500)

In [ ]:
print("\nOptuna Best Trial:")
best = studyRFC.best_trial
print(f"Validation Loss: {best.value:.4f}")
for key, val in best.params.items():
    print(f"  {key}: {val}")
print("\n")

best_n_estimators = best.params["n_estimators"]
best_criterion = best.params["criterion"]
best_max_depth = best.params["max_depth"]
best_max_features = best.params["max_features"]
best_random_state = best.params["random_state"]
best_warm_start = best.params["warm_start"]

In [ ]:
finalModel = RandomForestClassifier(n_estimators = best_n_estimators,
                                    criterion = best_criterion,
                                    max_depth = best_max_depth,
                                    max_features = best_max_features,
                                    random_state = best_random_state, 
                                    warm_start = best_warm_start)

In [ ]:
joblib.dump(finalModel, "savedModel/rfcmodel.joblib")

In [ ]:
# model = RandomForestClassifier(n_estimators = 10, max_features = 4, random_state = 101)

In [ ]:
# model.fit(X_train_text,Y_train_labels)

In [ ]:
# predictions = model.predict(X_test_text)

### Evaluation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
confusion_mat = multilabel_confusion_matrix(Y_test_labels, predictions)

In [ ]:
def accuracy(tp, tn, fp, fn):
    return (tp + tn) / (tp + tn + fp + fn)


def precision(tp, fp):
    return tp / (tp + fp)


def recall(tp, fn):
    return tp / (tp + fn)

def f1_score(tp,fp,fn):
    precision_val = precision(tp,fp)
    recall_val = recall(tp,fn)
    return 2 * (precision_val * recall_val) / (precision_val + recall_val)

In [ ]:
def print_eval(title, accuracy, precision, recall, f1_score):
    print(f"{title}\n")
    print(f"accuracy: {round(accuracy,2)}")
    print(f"precision: {round(precision,2)}")
    print(f"recall: {round(recall,2)}")
    print(f"f1 Score: {round(f1_score,2)}")
    print("=======\n")


def present_data(log_level):
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for i in range(0, len(confusion_mat)):
        tp, fp = confusion_mat[i][0]
        fn, tn = confusion_mat[i][1]

        accuracy_val = accuracy(tp, tn, fp, fn)
        precision_val = precision(tp, fp)
        recall_val = recall(tp, fn)
        f1_score_val = f1_score(tp, fp, fn)

        total_accuracy += accuracy_val
        total_precision += precision_val
        total_recall += recall_val
        total_f1_score += f1_score_val

        if(log_level == "emotions"):
            print_eval(
            f"Emotion: {emotions[i]}",
                accuracy_val,
                precision_val,
                recall_val,
                f1_score_val,
            )

    avg_accuracy = total_accuracy / len(confusion_mat)
    avg_precision = total_precision / len(confusion_mat)
    avg_recall = total_recall / len(confusion_mat)
    avg_f1_score = total_f1_score / len(confusion_mat)

    if(log_level=="macro"):
        print_eval("Macro Average:", avg_accuracy, avg_precision, avg_recall, avg_f1_score)

In [ ]:
present_data("emotions")

In [ ]:
present_data("macro")